In [ ]:
import numpy as np
from glob import glob
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

In [ ]:
source_dir = "C:/Users/shakib.mahmud/Desktop/federated RBC/train"
val_dir = "C:/Users/shakib.mahmud/Desktop/federated RBC/val"
test_dir = "C:/Users/shakib.mahmud/Desktop/federated RBC/test"
client_dirs = glob(source_dir+ "/*")

In [ ]:
client_dirs

In [ ]:
input_shape = (128, 128)
batch_size = 32

generators_train_client = []
generators_val_client = []

for i in range(0, len(client_dirs)):
    datagen_train = ImageDataGenerator(rescale=1./255, validation_split=0.1)
    generator_train = datagen_train.flow_from_directory(directory=client_dirs[i],
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True)
    generator_val = datagen_train.flow_from_directory(directory=client_dirs[i],
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True)
    generators_train_client.append(generator_train)
    generators_val_client.append(generator_val)
num_classes = generators_train_client[0].num_classes

In [ ]:
datagen_test = ImageDataGenerator(rescale=1./255)
generators_test = datagen_train.flow_from_directory(directory=val_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=False)

In [ ]:
class SimpleModel:
    def build(self):
        model = VGG16(include_top=False,input_shape=(128, 128, 3), weights='imagenet')
        transfer_layer = model.get_layer('block5_pool')
        conv_model = Model(inputs=model.input, outputs=transfer_layer.output)
        # Start a new Keras Sequential model.
        new_model = Sequential()

        # Add the convolutional part of the VGG16 model from above.
        new_model.add(conv_model)

        # Flatten the output of the VGG16 model because it is from a
        # convolutional layer.
        new_model.add(Flatten())

        # Add a dense (aka. fully-connected) layer.
        # This is for combining features that the VGG16 model has
        # recognized in the image.

        new_model.add(Dense(1024, activation='relu'))


        # Add the final layer for the actual classification.
        new_model.add(Dense(num_classes, activation='softmax'))

        return new_model

In [ ]:
opt = Adam(learning_rate=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

In [ ]:
smlp_global = SimpleModel()
global_model = smlp_global.build()
global_model.compile(optimizer= opt, loss=loss, metrics=metrics)

local_model = smlp_global.build()
local_model.compile(optimizer= opt, loss=loss, metrics=metrics)

In [ ]:
global_model.predict(generators_test)

In [ ]:
def scale_model_weights(weight, scalar):
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

In [ ]:
# validation_data=(x_test, y_test)
comms_round = 1
comm_round_accs = []
comm_round_losses = []
for comm_round in range(comms_round):
    global_weights = global_model.get_weights()
    scaled_local_weight_list = []

    for client in range(0, len(generators_train_client)):
        local_model.set_weights(global_weights)
        steps_per_epoch = generators_train_client[client].n / batch_size
        steps_val = generators_val_client[client].n / batch_size
        print(steps_val)
        history = local_model.fit_generator(generator=generators_train_client[client],
                                  epochs=1,
                                  validation_data=generators_val_client[client],
                                  validation_steps=steps_val)

        scaled_local_weights = scale_model_weights(local_model.get_weights(), 1/len(client_dirs))
        scaled_local_weight_list.append(scaled_local_weights)

    average_weights = sum_scaled_weights(scaled_local_weight_list)
    global_model.set_weights(average_weights)

    scores = global_model.evaluate_generator(generators_test)
    print("Communication round " + str(comm_round)+ " accuracy is "+str(scores[1]*100))
    print("Communication round " + str(comm_round)+ " loss is "+str(scores[0]))
    comm_round_accs.append(scores[1]*100)
    comm_round_losses.append(scores[0])
    #print("Communication round " + str(comm_round)+ " accuracy is "+scores)

In [ ]:
generators_test = datagen_train.flow_from_directory(directory=test_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=False)

In [ ]:
predict = global_model.predict(generators_test)

In [ ]:
predict=np.argmax(predict,axis=1)

In [ ]:
generators_train_client[0].classes

In [ ]:
comm_round_accs

In [ ]:
comm_round_losses

In [ ]:
import matplotlib.pyplot as plt
plt.plot(comm_round_losses)

plt.title('Global Model Loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(comm_round_accs)


plt.title('Global Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

rounds = range(1, len(comm_round_accs) + 1)

plt.figure(figsize=(8, 5))

plt.plot(rounds, comm_round_accs, label='Accuracy (%)', color='blue', linewidth=2)
plt.plot(rounds, comm_round_losses, label='Loss', color='orange', linewidth=2)

plt.title('Global Model Learning Curves')
plt.xlabel('Communication Round')
plt.ylabel('Scores')

plt.legend()
plt.grid(True)

plt.show()


In [ ]:
comm_round_accs

In [ ]:
scores = global_model.evaluate(generators_test)

In [ ]:
predict.shape

In [ ]:
generators_test.classes.shape

In [ ]:
from sklearn.metrics import classification_report
import seaborn as sns
import pandas as pd

clf_report = classification_report(
    generators_test.classes,
    predict,
    target_names=[
        "Gallstones",
        "Abdomen and retroperitoneum",
        "cholecystitis",
        "Membranous and gangrenous cholecystitis",
        "Perforation",
        "Polyps and cholesterol crystals",
        "Adenomyomatosis",
        "Carcinoma",
        "Various causes of gallbladder wall thickening"
    ],
    digits=4
)
print(clf_report)

clf_report = classification_report(
    generators_test.classes,
    predict,
    target_names=[
        "Gallstones",
        "Abdomen and retroperitoneum",
        "cholecystitis",
        "Membranous and gangrenous cholecystitis",
        "Perforation",
        "Polyps and cholesterol crystals",
        "Adenomyomatosis",
        "Carcinoma",
        "Various causes of gallbladder wall thickening"
    ],
    output_dict=True
)

sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True, cmap="GnBu")


In [ ]:
from sklearn.metrics import confusion_matrix
array = confusion_matrix(generators_test.classes, predict)
df_cm = pd.DataFrame(array, index = ["acanthocytes","codocytes","elliptocyte","hypochromic","normal","spherocytes","stomatocyte","dacrocytes"],
                  columns = ["acanthocytes","codocytes","elliptocyte","hypochromic","normal","spherocytes","stomatocyte","dacrocytes"])
plt.figure(figsize = (6,5))
cm_plot = sns.heatmap(df_cm, annot=True, cmap='GnBu', fmt='g')
cm_plot.set_xlabel('Predicted Labels')
cm_plot.set_ylabel('True Labels')
cm_plot.set_title('Confusion Matrix From Federated Learning', size=12)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

array = confusion_matrix(generators_test.classes, predict)
df_cm = pd.DataFrame(array,
                     index=["Gallstones",
                            "Abdomen and retroperitoneum",
                            "cholecystitis",
                            "Membranous and gangrenous cholecystitis",
                            "Perforation",
                            "Polyps and cholesterol crystals",
                            "Adenomyomatosis",
                            "Carcinoma",
                            "Various causes of gallbladder wall thickening"],
                     columns=["Gallstones",
                              "Abdomen and retroperitoneum",
                              "cholecystitis",
                              "Membranous and gangrenous cholecystitis",
                              "Perforation",
                              "Polyps and cholesterol crystals",
                              "Adenomyomatosis",
                              "Carcinoma",
                              "Various causes of gallbladder wall thickening"])
plt.figure(figsize = (6,5))
cm_plot = sns.heatmap(df_cm, annot=True, cmap='GnBu', fmt='g')
cm_plot.set_xlabel('Predicted Labels')
cm_plot.set_ylabel('True Labels')
cm_plot.set_title('Confusion Matrix From Federated Learning', size=12)
